<a href="https://colab.research.google.com/github/UdayLab/PAMI/blob/main/notebooks/airPollutionAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>